In [1]:
%load_ext autoreload
%autoreload 2

import os
from dotenv import load_dotenv
load_dotenv()

from clients.prompt_client import PromptClient
import json

client = PromptClient(
    base_url=f"{os.getenv('BASE_URL')}/{os.getenv('APPLICATION_NAME')}/v1",
    api_key=os.getenv('API_TOKEN')
)

## First, launch the server, according to whats written in the readme.

## Understanding prompt structure

In [2]:
text, objects, tools = client.call(
    input_data={'history':[], 'persona': 'A llm engineer willing to learn'},
    route="/prompts/readme"
)

print(text)

Great! Let me explain how prompt files work in a way that should be helpful for you as an LLM engineer.

### Overview of Prompt Files

Prompt files are a structured way to define complex prompts for language models using a markdown file format. This makes it easier to read, iterate, and manage your prompts. The structure is divided into sections using triple dashes (`---`), with the first section always being the configuration.

### Configuration Section

The configuration section is a YAML object that must include the following fields:

1. **Mirascope Call Params**: Parameters for the Mirascope call. You can refer to the [Mirascope documentation](https://mirascope.com/docs/mirascope/learn/calls) for more details.

2. **Model**: You can define a model here using Litellm syntax or use one of the standard models defined in environment variables.

3. **Stream**: Whether to stream the response (always set to `true`).

4. **Json Mode**: Whether to use JSON mode (not supported yet).

5. **To

## Of course, you can also stream the response

In [3]:
response = client.stream(
    input_data={'history':[], 'persona': 'A llm engineer willing to learn'},
    route="/prompts/readme"
)

for elt in response:
    if elt['format'] == 'text':
        print(elt['content'], end="")

Great! Let me explain how prompt files work in a way that should be helpful for you as an LLM engineer.

### Overview of Prompt Files

Prompt files are a structured way to define complex prompts for language models using a markdown file format. This makes it easier to read, iterate, and manage your prompts. The structure is divided into sections using triple dashes (`---`), with the first section always being the configuration.

### Configuration Section

The configuration section is a YAML object that must include the following fields:

1. **Mirascope Call Params**: Parameters for the Mirascope call. You can refer to the [Mirascope documentation](https://mirascope.com/docs/mirascope/learn/calls) for more details.

2. **Model**: You can define a model here using Litellm syntax or use one of the standard models defined in environment variables.

3. **Stream**: Whether to stream the response (always set to `true`).

4. **Json Mode**: Whether to use JSON mode (not supported yet).

5. **To

## Tool call? Easy

Tools are defined as python function in /src/tools

build a simple square tool, to define it, we create the file /src/tools/calculator

```python
async def square(n: float) -> float:
    """Takes in a number n, returns the square of n

    Args:
       n (float): The number to be squared.
    """
    return n**2
```

then simply add it to the export in the __init__.py of the tool folder

```python
from src.tools.calculator import square
from src.tools.llm_planner import llm_planner
from src.tools.retrieval import search_movie_bm25

__all__ = ["square", "llm_planner", "search_movie_bm25"]
```

then restart the server, and you can define the tool directly by referencing it in your prompt.

Case in point : the square prompt in /prompts/square.md

In [4]:
response = client.stream(
    input_data={'history':[], 'number': 4},
    route="/prompts/square"
)

for elt in response:
    if elt['format'] == 'text':
        print(elt['content'], end="")
    elif elt['format'] == "tool":
        print("************ TOOL CALL ************")
        print(elt['content'])
        print("************")

************ TOOL CALL ************
<tool_call>
### NAME: square
### ARGUMENTS: ("n": 4.0)
### RESULT: 16.0
</tool_call>
************
The square of 4.0 is 16.0

## Streaming objects

In many cases, you can find yourself needing to structure json or yaml with a llm.
Instead of using the json mode, the idea here is to be able to parse directly objects from the stream, enabling more interactivity

In [5]:
response = client.stream(
    input_data={'history':[], 'number': 4},
    route="/prompts/square_object"
)

for elt in response:
    if elt['format'] == 'text':
        print(elt['content'], end="")
    elif elt['format'] == "tool":
        print("\n************ TOOL CALL ************")
        print(elt['content'])
        print("*************************************\n")
    elif elt['format'] == "object":
        print("\n************ Object parsed ************")
        print(json.loads(elt['content']))
        print("*************************************\n")


************ TOOL CALL ************
<tool_call>
### NAME: square
### ARGUMENTS: ("n": 4.0)
### RESULT: 16.0
</tool_call>
*************************************

```yaml
input: 4.0
output: 16.0
rational: The square of 4.0 is calculated by multiplying 4.0 by itself, resulting in 16.0.

************ Object parsed ************
{'input': '4.0', 'output': '16.0', 'rational': 'The square of 4.0 is calculated by multiplying 4.0 by itself, resulting in 16.0.'}
*************************************

```

I have squared the number 4.0 and obtained the result 16.0.

## Wanna batch? Lets go

In [6]:
sentence_to_translate = [
    {"target_language":"french", "text":"It's raining cats and dogs"},
    {"target_language":"spanish", "text":"The sun is shining brightly"},
    {"target_language":"german", "text":"I love playing football"},
    {"target_language":"italian", "text":"The pizza is delicious"},
    {"target_language":"portuguese", "text":"The beach is beautiful"},
]

results = client.batch(
    input_data=sentence_to_translate,
    route="/prompts/translate",
    n_jobs=5,
    verbose=50
)

results

[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done   1 tasks      | elapsed:    1.6s
[Parallel(n_jobs=5)]: Done   2 out of   5 | elapsed:    1.7s remaining:    2.5s
[Parallel(n_jobs=5)]: Done   3 out of   5 | elapsed:    1.7s remaining:    1.1s
[Parallel(n_jobs=5)]: Done   5 out of   5 | elapsed:    2.4s finished


[('Il pleut des cordes', [], []),
 ('El sol brilla con fuerza', [], []),
 ('Ich liebe Fußball zu spielen', [], []),
 ('La pizza è deliziosa', [], []),
 ('A praia é bonita', [], [])]

## Bringing vision

Prompt server is natively multimodal, just send your url or base64 data, use a compatible provider, and you're good

Example: let's build and advanced flux Kontext prompt

<div style="text-align:center;">
  <img src="https://www.science.org/do/10.1126/science.aaw4076/full/dolphin_16x9_2-1644908628810.jpg" alt="Dolphin" style="max-width:512px;">
</div>

In [8]:
image_url = "https://www.science.org/do/10.1126/science.aaw4076/full/dolphin_16x9_2-1644908628810.jpg"
edit_instruction="I want to make a logo based on this image"


response = client.stream(
    input_data={'history':[], 'image': image_url, 'prompt':edit_instruction},
    route="/prompts/kontext"
)

for elt in response:
    if elt['format'] == 'text':
        print(elt['content'], end="")
    elif elt['format'] == "tool":
        print("\n************ TOOL CALL ************")
        print(elt['content'])
        print("*************************************\n")
    elif elt['format'] == "object":
        print("\n************ Object parsed ************")
        print(json.loads(elt['content']))
        print("*************************************\n")

### 1. UNDERSTAND: Carefully analyze the simple instruction provided by the user. Identify the main objective and any specific details mentioned.

The main objective is to create a logo based on the provided image of two dolphins leaping out of the water. The instruction is quite simple and lacks specific details about the desired style, composition, or elements to preserve or modify.

### 2. DESCRIPTION: Use the description of the base image to provide context for the modifications. This helps in understanding what elements need to be preserved or changed.

The base image features two dolphins leaping out of the water. The dolphins are in mid-air, with water splashing around them. The background is a clear blue ocean with a light blue sky. The dolphins are the central focus of the image, and their dynamic pose suggests movement and energy.

### 3. DETAILS: If the users instruction is vague, use creative imagination to infer necessary details. This may involve expanding on the instruct

## Promptception : prompt as tool

The tool llm planner is an example of how to use a prompt as tool inside an other prompt

In [20]:
response = client.stream(
    input_data={'history':[], 'number': 4},
    route="/prompts/double_square_plan"
)

for elt in response:
    if elt['format'] == 'text':
        print(elt['content'], end="")
    elif elt['format'] == "tool":
        print("\n************ TOOL CALL ************")
        print(elt['content'])
        print("*************************************\n")
    elif elt['format'] == "object":
        print("\n************ Object parsed ************")
        print(json.loads(elt['content']))
        print("*************************************\n")


************ TOOL CALL ************
<tool_call>
### NAME: llm_planner
### ARGUMENTS: ("tools": "[(\"name\": \"square\", \"tool\": (\"square\": (\"n\": (\"description\": \"The number to be squared.\"))))]", "task": "Compute the power four of a number provided by the user, using all the tools at your disposal")
### RESULT: ```plan
1. Perform action 1 using square
2. Perform action 2 using square
Provide the final answer
```
</tool_call>
*************************************


************ TOOL CALL ************
<tool_call>
### NAME: square
### ARGUMENTS: ("n": 4)
### RESULT: 16.0
</tool_call>
*************************************


************ TOOL CALL ************
<tool_call>
### NAME: square
### ARGUMENTS: ("n": 16)
### RESULT: 256.0
</tool_call>
*************************************

The power four of 4.0 is 256.0.